# 📊 Forecasting and Hedging Market Risk
## A GARCH-Based Value-at-Risk Analysis of the S&P 500
**Author:** Adam Saad
**Data Source:** Stooq via Excel

The model applies a rolling GARCH(1,1) with t-distributed errors, evaluates Value-at-Risk (VaR) forecasts, and validates performance using Kupiec’s backtest.

## 📚 References
- *Financial Risk Forecasting* by Jon Danielsson
- *Measuring Market Risk* by Kevin Dowd

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from arch import arch_model
from scipy.stats import t, chi2

In [2]:
# Load SPX data from Excel
data = pd.read_excel("data/SPX_data_stooq.xlsx", index_col="Date", parse_dates=True)
data['log_return'] = np.log(data['Close'] / data['Close'].shift(1))
returns = data['log_return'].dropna() * 100

In [9]:
rolling_var_95 = []
rolling_var_99 = []
window_size = 1000

for idx in range(window_size, len(returns)):
    window = returns[idx - window_size:idx]
    model = arch_model(window, vol='GARCH', p=1, q=1, dist='t')
    res = model.fit(disp='off')
    forecast = res.forecast(horizon=1)
    cond_var = forecast.variance.iloc[-1, 0]
    t_df = res.params['nu']
    mu = res.params['mu']
    sigma = np.sqrt(cond_var)
    rolling_var_95.append(mu + sigma * t.ppf(0.05, df=t_df))
    rolling_var_99.append(mu + sigma * t.ppf(0.01, df=t_df))

test_returns = returns[window_size:]
VaR_95_series = pd.Series(rolling_var_95, index=test_returns.index)
VaR_99_series = pd.Series(rolling_var_99, index=test_returns.index)

In [11]:
# Kupiec test function
def kupiec_test(returns, var_series, alpha):
    exceed = returns < var_series
    T = len(returns)
    x = exceed.sum()
    pi = x / T
    LR_uc = -2 * (np.log(((1 - alpha) ** (T - x)) * (alpha ** x)) -
                  np.log(((1 - pi) ** (T - x)) * (pi ** x)))
    p_value = 1 - chi2.cdf(LR_uc, df=1)
    return x, LR_uc, p_value

x95, LR95, p95 = kupiec_test(test_returns, VaR_95_series, 0.05)
x99, LR99, p99 = kupiec_test(test_returns, VaR_99_series, 0.01)

print(f"Kupiec Test 95%: Exceedances={x95}, LR={LR95:.2f}, p-value={p95:.3f}")
print(f"Kupiec Test 99%: Exceedances={x99}, LR={LR99:.2f}, p-value={p99:.3f}")

Kupiec Test 95%: Exceedances=100, LR=12.51, p-value=0.000
Kupiec Test 99%: Exceedances=22, LR=1.29, p-value=0.257


In [12]:
# Plotly chart for VaR and returns
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_returns.index, y=test_returns, mode='lines', name='S&P 500 Return',
                         line=dict(color='black'),
                         hovertemplate='Return: %{y:.2f}%<br>Date: %{x|%Y-%m-%d}<extra></extra>'))
fig.add_trace(go.Scatter(x=VaR_95_series.index, y=VaR_95_series, mode='lines', name='VaR 95%',
                         line=dict(dash='dash'),
                         hovertemplate='VaR 95%: %{y:.2f}%<br>Date: %{x|%Y-%m-%d}<extra></extra>'))
fig.add_trace(go.Scatter(x=VaR_99_series.index, y=VaR_99_series, mode='lines', name='VaR 99%',
                         line=dict(dash='dot'),
                         hovertemplate='VaR 99%: %{y:.2f}%<br>Date: %{x|%Y-%m-%d}<extra></extra>'))
exceedances = test_returns < VaR_99_series
fig.add_trace(go.Scatter(x=test_returns[exceedances].index, y=test_returns[exceedances], mode='markers',
                         name='Exceedances (99%)', marker=dict(color='red', size=6, symbol='x'),
                         hovertemplate='Exceedance: %{y:.2f}%<br>Date: %{x|%Y-%m-%d}<extra></extra>'))
fig.update_layout(autosize=True, height=600, template='plotly_white',
                  margin=dict(l=20, r=20, t=50, b=40),
                  legend=dict(x=0, y=1.1, orientation='h'),
                  title='S&P 500 Daily Returns with GARCH-Based VaR',
                  xaxis_title='Date', yaxis_title='Return (%)')
fig.show()